<a href="https://colab.research.google.com/github/PyThaiNLP/thaimaimeex/blob/master/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ThaiME Project Name Generation

This notebook allows you to generate ThaiME project names based on the language model trained on over 30,000 ThaiME project names. You can also try naming your own project and have it estimated by a LinearSVR model trained on the [thaimaimeex](https://github.com/PyThaiNLP/thaimaimeex) dataset.

## Run Below to Prepare

In [0]:
#download and installs for colab
!npx degit pythainlp/thaimaimeex -f
!pip install pythainlp -q
!pip install emoji

#download files
#budget estimator
!wget https://github.com/PyThaiNLP/pythainlp-corpus/releases/download/thaime_lm_lstm-v.01/thaime_clf.joblib
#lm data
!wget https://github.com/PyThaiNLP/pythainlp-corpus/releases/download/thaime_lm_lstm-v.01/thaime_lm.pkl
#lm model
!wget https://github.com/PyThaiNLP/pythainlp-corpus/releases/download/thaime_lm_lstm-v.01/thaime_lm.pth
!mkdir models; mv thaime_lm.pth models

#imports
import numpy as np
from joblib import dump, load

#fastai
import fastai
from fastai.text import *

#classification model
clf = load('thaime_clf.joblib') 

#generation model
data_lm = load_data('','thaime_lm.pkl')
data_lm.sanity_check()

#check vocab size
len(data_lm.vocab.itos)

config = dict(emb_sz=400, n_hid=1550, n_layers=4, pad_token=1, qrnn=False, tie_weights=True, out_bias=True,
             output_p=0.25, hidden_p=0.1, input_p=0.2, embed_p=0.02, weight_p=0.15)
trn_args = dict(drop_mult=0.9, clip=0.12, alpha=2, beta=1)

learn = language_model_learner(data_lm, AWD_LSTM, config=config, pretrained=False, **trn_args)

#load pretrained models
learn.load('thaime_lm');

## Generate Project Names

Generate project names with some parameters.

In [11]:
#@title {run: "auto"}
Length = 25 #@param {type:"slider", min:10, max:200, step:1}
Temperature = 0.8 #@param {type:"slider", min:0.1, max:1.5, step:0.1}
Seed_Word = 'โครงการ' #@param {type:"string"}
nb_ = 20

for i in range(nb_):
    t = learn.predict(Seed_Word, 
                      Length, 
                      temperature=Temperature, 
                      min_p=0.005, sep = '')
    est = np.round(np.exp(clf.predict([t])[0])/1e6,2)
    print(f'{t.replace("xxbos","")} ราคา:{est}M')

โครงการเสริมผิวทางหินคลุกสายทางการเกษตรดงนาคำ-นาคู โครงการก่อสร้างถนนคอนกรีตเสริมเหล็ก หมู่ที่ 3 ราคา:0.37M
โครงการก่อสร้างถนน คสล. สายบ้านนายทวี ถึงบ้านนายสมจิตร สุขแก้ว หมู่ที่ 12  ราคา:0.91M
โครงการก่อสร้างถนนคอนกรีตเสริมเหล็ก สายข้างวัดพระแท่น - วัดเนินน้ำ ตำบลห้วยโป่ง อำเภอเมืองพัทลุง จังหวัด ราคา:1.27M
โครงการก่อสร้างถนนคอนกรีตเสริมเหล็ก สายคลองหาด หมู่ที่ 4 บ้านคลองตา ตำบลบางรัก อำเภอคลองหลวง  ราคา:1.16M
โครงการก่อสร้างถนนคอนกรีตเสริมเหล็ก หมู่ 4 บ้านท่าช้าง โครงการก่อสร้าง ผิวจราจรแบบ asphaltic concrete สายทาง ราคา:2.64M
โครงการก่อสร้างถนน คสล. สายทางเข้าห้วยยางทอง บ้านนาห้วยใหญ่ หมู่ที่ 1 ตำบลน้ำอ้อม ราคา:1.11M
โครงการขุดลอกคลอง หมู่ที่ 6 บ้านหนองตะเคียน ก่อสร้างถนนคอนกรีตเสริมเหล็ก สายหนองหลวง หมู่ 1 ราคา:0.37M
โครงการยกระดับผลิตภัณฑ์และการบริการทางการตลาดและการตลาดเพื่อเพิ่มมูลค่าให้แก่ตลาดสินค้าเกษตรและการตลาด โครงการพัฒนาเส้นทางขนส่ง ราคา:5.13M
โครงการก่อสร้างถนนคอนกรีตเสริมเหล็ก / สายบ้านนางสุรี - หนองนา บ้านท่าไม้ หมู่ที่ 12 ราคา:0.59M
โครงการก่อสร้างถนนคอนกรีตเสริมเหล็ก

## DIY Project Names

Enter your own project names and see how much budget it would have.

In [24]:
#@title {run: "auto"}
Project_Name = '\u0E42\u0E04\u0E23\u0E07\u0E01\u0E32\u0E23 blockhain AI Iot \u0E2D\u0E30\u0E44\u0E23\u0E2A\u0E31\u0E01\u0E2D\u0E22\u0E48\u0E32\u0E07' #@param {type:"string"}
cost = round(np.exp(clf.predict([Project_Name])[0])/1e6,2)
print(f'{cost}M THB')

5.39M THB
